In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


In [ ]:
parcel_df = pd.read_csv('Parcel.csv', encoding='latin-1')

In [ ]:
parcel_df= parcel_df.set_index(keys=['Major', 'Minor'])

In [ ]:
# Only take KING COUNTY properties from parcel.csv
parcel_df = parcel_df[parcel_df['DistrictName']=='KING COUNTY']

In [ ]:
# parcel_df.head()

In [ ]:
parcel_df = parcel_df[["SqFtLot", "WfntLocation"]]

In [ ]:
# parcel_df.head()

In [ ]:
residentail_blg_df = pd.read_csv('ResBldg.csv')

In [ ]:
residentail_blg_df.columns
residentail_blg_df = residentail_blg_df[['Major', 'Minor', 'Address', 'SqFtTotLiving', 'Bedrooms']]
# residentail_blg_df.head()

In [ ]:
sale_df = pd.read_csv('Sale.csv')


In [ ]:
sale_df.head()


In [ ]:
sale_parcel_df = pd.merge(sale_df, parcel_df, right_index=True, left_on=['Major', 'Minor'])


In [ ]:
all_df = pd.merge(sale_parcel_df, unit_df, right_index=True, left_on=['Major', 'Minor'])

In [ ]:
unit_df.shape


In [ ]:
sale_df.columns
# removing the 2 rows without Major+Minor index


In [ ]:
sale_df['Major'] = pd.to_numeric(sale_df['Major'], errors='coerce')
sale_df['Minor'] = pd.to_numeric(sale_df['Minor'], errors='coerce')



In [ ]:
sale_df = sale_df.dropna()


In [ ]:
sale_df.isna().sum()


In [ ]:
sale_df = sale_df.set_index(keys=['ExciseTaxNbr','Major', 'Minor'])

sale_df.head()


In [ ]:
parcel_df.head()


In [ ]:
sale_df.isna().sum()


In [ ]:
sale_df.info()


In [ ]:
sale_df.head(1)


In [ ]:
parcel_df


In [ ]:
unit_df.info()



In [ ]:
unit_df.head()

In [ ]:
unit_df['NbrBedrooms'].astype(int)

In [ ]:
#assuming S means Studio
nbr = unit_df['NbrBedrooms']


In [ ]:
nbr = nbr.str.replace('S', '0')


In [ ]:
#Replace spaces with 0
nbr = nbr.str.replace(' ', '0')


In [ ]:
nbr = nbr.astype(float)


In [ ]:
unit_df['NbrBedrooms'] = nbr


In [ ]:
unit_df.info()


In [ ]:
# what im going to do
wftn = parcel_df['WfntLocation']


In [ ]:
wftn = wftn.astype(bool)


In [ ]:
parcel_df['WfntLocation'] =wftn


In [ ]:
parcel_df.info()


In [ ]:
parcel_df.head(5)



In [ ]:
unit_df.head(5)



In [ ]:
parcel_unit_df = pd.merge(parcel_df, unit_df, how='inner', left_index=True, right_index=True)

In [ ]:
parcel_unit_df.info()


In [ ]:
sale_df.head()
sale_df.reset_index(level=0,inplace=True)

In [ ]:
sale_df


In [ ]:
unit_price_df = pd.merge(sale_df, unit_df, how='inner', left_index=True , right_index=True)

In [ ]:
# YAY! the inner join worked
unit_price_df

In [ ]:
# this doesnt work
housing_df = pd.merge(unit_price_df, parcel_df, how='left', left_index=True , right_index=True)

In [ ]:
housing_df.info()


In [ ]:
sale_df = sale_df[['DocumentDate','SalePrice', 'PropertyClass']]


In [ ]:
sale_df


In [ ]:
new_df = pd.merge(sale_df, residentail_blg_df, left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

In [ ]:
new_df.head()


In [ ]:
parcel_res_df =pd.merge(parcel_df, residentail_blg_df, left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

In [ ]:
parcel_res_df.shape

In [ ]:
housing_df = pd.merge(parcel_df, new_df, left_on=['Major', 'Minor'], right_on=['Major', 'Minor'])

In [ ]:
housing_df.shape

In [ ]:
housing_df.columns

## We are only looking at the 2017-2018 documented data

In [ ]:
housing = housing_df[(housing_df['DocumentDate'].str.endswith('2018'))|(housing_df['DocumentDate'].str.endswith('2017'))]
# housing.info()

## select the features

In [ ]:
housing.columns

In [ ]:
housing = housing[['Major','Minor','SqFtLot','WfntLocation','SalePrice','Bedrooms']]

In [ ]:
# housing.info()

## change the waterfront values into binary values and drop the 0 saleprice 

In [ ]:
housing.loc[housing['WfntLocation']!=0,'WfntLocation']=1
housing.WfntLocation.unique()

In [ ]:
drop_0sale = housing[housing['SalePrice']==0]
housing.drop(drop_0sale.index,axis=0,inplace=True)

In [ ]:
housing.SalePrice.unique()

## slice only the 3th to 95th percentile of the saleprice

In [ ]:
# let's check the distribution of the saleprice
import seaborn as sns
fig,ax = plt.subplots(figsize=(7,7))
ax = sns.distplot(housing.SalePrice)
ax.set_title('The distribution of the saleprice (before the slicing)')

In [ ]:
# Find the 3th of the saleprice and the 95th of the salsprice
third = np.quantile(housing.SalePrice,.03)
ninty_five = np.quantile(housing.SalePrice,.95)
print(third)
print(ninty_five)

In [ ]:
np.__version__

In [ ]:
# After sorted the saleprice, we get rid of the last 5%

# sorted_housing = housing.SalePrice.sort_values(axis=0)
# fifth_of_price = sorted_housing[round(0.05*len(sorted_housing))]
# print('These are the original size of the dataframe: ',format(len(sorted_housing)))
# sorted_toolow_housing = sorted_housing[:round(0.05*len(sorted_housing))]
# print('These are the first 5% of the dataframe: ',format(len(sorted_toolow_housing)))
# sorted_toohigh_housing = sorted_housing[round(-0.05*len(sorted_housing)):]
# print('These are the last 5% of the dataframe: ',format(len(sorted_toohigh_housing)))
# housing_df = housing.drop(index=sorted_toolow_housing.index)
# housing_df = housing.drop(index=sorted_toohigh_housing.index)
# print('These are the final size of the dataframe: ',format(len(housing_df)))
# print(min(housing_df.SalePrice))
housing_df = ((housing_df.loc[(housing_df['SalePrice']<ninty_five)&(housing_df['SalePrice']>third)]))
# housing_df = (housing_df.loc[housing_df['SalePrice']<ninty_five])



In [ ]:
fig,ax = plt.subplots(figsize=(7,7))
ax = sns.distplot(housing_df.SalePrice)
ax.set_title('The distribution of the saleprice (after the sclicing)')

## Normalizing the saleprice

In [ ]:
housing_df.SalePrice = np.log(housing_df.SalePrice)

In [ ]:
fig,ax = plt.subplots(figsize=(7,7))
ax = sns.distplot(housing_df.SalePrice)
ax.set_title('The distribution of the saleprice (after the sclicing and scaling)')

## check the multicolinearity 

In [ ]:
predictors = housing_df[['WfntLocation','Bedrooms','SqFtLot']]

In [ ]:
pd.plotting.scatter_matrix(predictors,figsize=(7,7));

In [ ]:
sns.heatmap(predictors.corr(),center=0)

In [ ]:
housing.SqFtLot.unique()

## The multiple linear regression using Statsmodels

In [ ]:
import statsmodels.api as sm
from statsmodels.formula.api import ols
predictors_int = sm.add_constant(predictors)
model = sm.OLS(housing_df.SalePrice,predictors_int).fit()

In [ ]:
model.params

In [ ]:
model.summary()

## The multiple linear regression using sklearn

In [ ]:
from sklearn.linear_model import LinearRegression
y = housing_df.SalePrice
linreg = LinearRegression()
linreg.fit(predictors,y)

In [ ]:
print(linreg.coef_)
print(linreg.intercept_)

## Cross-validation

In [ ]:
from sklearn.model_selection import train_test_split
import sklearn.metrics as metrics
X_train, X_test, y_train, y_test = train_test_split(predictors, housing_df.SalePrice)
lr = LinearRegression()
lr.fit(X_train, y_train)
lr.predict(X_test)
print(lr.score(X_test, y_test))
print(lr.coef_)
print(lr.intercept_)


## Let's see if the prediction model would be better if we use KNN 

## We are also doing the standard scaling

In [ ]:
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

In [ ]:
scaler = StandardScaler()

scaler.fit(X_train)

X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Let's use the sklearn's KNN implementation

In [ ]:
from sklearn.neighbors import KNeighborsRegressor

In [ ]:
nn = KNeighborsRegressor(n_neighbors=5, n_jobs=-1)

nn.fit(X_train_scaled, y_train)

sk_preds = nn.predict(X_test_scaled)

rmse = np.sqrt(mean_squared_error(y_test, sk_preds))

print(f'Root Mean Squared Error: {rmse}')

## What K value should we pick?

In [ ]:
ks = range(1, 30)

test_errors = np.zeros(len(list(ks)))

for i, k in enumerate(ks):
    
    nn = KNeighborsRegressor(n_neighbors=k, n_jobs=-1)

    nn.fit(X_train_scaled, y_train)
    preds = nn.predict(X_test_scaled)
    
    test_errors[i] = np.sqrt(mean_squared_error(y_test, preds))

In [ ]:
fig, ax = plt.subplots(figsize=(10, 5))

ax.plot(list(ks), test_errors)
ax.axvline(list(ks)[np.argmin(test_errors)], linestyle='--', color='black');

In [ ]:
test_errors